<a href="https://colab.research.google.com/github/pacificspatial/flateau/blob/main/notebook/sdsc_bootcamp_tokyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DuckDB を使った Plateauデータの解析ハンズオン！

In [45]:
#!pip install leafmap
#!pip install pydeck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 12.9 MB/s eta 0:00:00


In [46]:
import pydeck
import leafmap
from shapely import wkt

In [1]:
# Duckdb is already in Colab!
import duckdb
import pandas as pd
import geopandas as gpd

## DuckDBが動くかテスト

In [2]:
# simple sql with an in-memory database
duckdb.sql('SELECT 42').show()

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [3]:
# check relation
r1 = duckdb.sql('SELECT 42 AS i')
duckdb.sql('SELECT i * 2 AS k FROM r1').show()

┌───────┐
│   k   │
│ int32 │
├───────┤
│    84 │
└───────┘



## Extension を確認

In [4]:
#con = duckdb.connect(database=":memory:", read_only=False, config={"allow_unsigned_extensions": "true"});
#con.execute('select * from duckdb_extensions()').fetch_df()
duckdb.sql('select * from duckdb_extensions()')

┌──────────────────┬─────────┬───────────┬──────────────┬──────────────────────────────────────────┬───────────────────┐
│  extension_name  │ loaded  │ installed │ install_path │               description                │      aliases      │
│     varchar      │ boolean │  boolean  │   varchar    │                 varchar                  │     varchar[]     │
├──────────────────┼─────────┼───────────┼──────────────┼──────────────────────────────────────────┼───────────────────┤
│ autocomplete     │ false   │ false     │              │ Add supports for autocomplete in the s…  │ []                │
│ fts              │ true    │ true      │ (BUILT-IN)   │ Adds support for Full-Text Search Inde…  │ []                │
│ httpfs           │ false   │ false     │              │ Adds support for reading and writing f…  │ [http, https, s3] │
│ icu              │ true    │ true      │ (BUILT-IN)   │ Adds support for time zones and collat…  │ []                │
│ inet             │ false   │ f

## Extensionをインストールして読み込む

In [5]:
duckdb.sql("INSTALL 'httpfs'");
duckdb.sql("INSTALL 'spatial'");

duckdb.sql('LOAD httpfs')
duckdb.sql('LOAD spatial')

## Geometryを取り扱えるか確認

In [6]:
duckdb.sql('SELECT ST_POINT(0,0)')

┌────────────────┐
│ st_point(0, 0) │
│    geometry    │
├────────────────┤
│ POINT (0 0)    │
└────────────────┘

## 今回使うデータのリスト

* building centroid
 * https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_centroid.parquet

* building footprint polygon
 * https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_polygon.parquet

* elevation
 * https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_elevation_h3lvl10.parquet

* slope
 * https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_slope_h3lvl10.parquet

* flooding
 * https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_risk_flooding.parquet

* high tide
 * https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_risk_high_tide.parquet

* landslide
 * https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_risk_land_slide.parquet

## まずデータを読み込む

In [18]:
# 建物2Dポリゴンの読み込み
duckdb.sql("drop table if exists building_polygon")
duckdb.sql("create table building_polygon as select id, gml_id, h3index10, cal_zmin_m, cal_height_m::double, ST_GeomFromWKB(geometry) as geom from 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_polygon.parquet' ")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [19]:
duckdb.sql('select * from building_polygon limit 3')

┌───────┬──────────────────────┬────────────────────┬────────────┬──────────────────────┬──────────────────────────────┐
│  id   │        gml_id        │     h3index10      │ cal_zmin_m │ CAST(cal_height_m …  │             geom             │
│ int64 │       varchar        │       uint64       │   double   │        double        │           geometry           │
├───────┼──────────────────────┼────────────────────┼────────────┼──────────────────────┼──────────────────────────────┤
│     1 │ bldg_fc50c7d9-76ac…  │ 622329813002289151 │      2.406 │    6.734999999999999 │ POLYGON ((139.712307088431…  │
│     2 │ bldg_2e11a57e-c315…  │ 622329810539020287 │      2.792 │                9.833 │ POLYGON ((139.707154711310…  │
│     3 │ bldg_72de059a-ddf0…  │ 622329810539118591 │      2.642 │                6.605 │ POLYGON ((139.706502252721…  │
└───────┴──────────────────────┴────────────────────┴────────────┴──────────────────────┴──────────────────────────────┘

In [9]:
# 標高データの読み込み（H3 level 10 のインデックスを含む）
duckdb.sql("drop table if exists elevation")
duckdb.sql("create table elevation as select h3index10, ST_GeomFromWKB(geometry) as geom from 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_elevation_h3lvl10.parquet'")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [10]:
# 斜面傾斜データの読み込み（H3 level 10 のインデックスを含む）
duckdb.sql("drop table if exists slope")
duckdb.sql("create table slope as select h3index10, val_max, val_mean, val_median, val_mode, ST_GeomFromWKB(geometry) as geom from 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_slope_h3lvl10.parquet'")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [20]:
# 建物が立っている地形の最大傾斜の大きい順に上位10件の建物のリストを取得
query_01 = 'select t1.gml_id, t1.geom, t2.val_max from building_polygon t1, slope t2 where t1.h3index10 = t2.h3index10 order by val_max desc limit 10'
duckdb.sql(query_01)

┌──────────────────────┬───────────────────────────────────────────────────────────────────────────┬───────────────────┐
│        gml_id        │                                   geom                                    │      val_max      │
│       varchar        │                                 geometry                                  │      double       │
├──────────────────────┼───────────────────────────────────────────────────────────────────────────┼───────────────────┤
│ bldg_26811f7d-0a49…  │ POLYGON ((139.74449723500115 35.66740033892029, 139.74445959311024 35.6…  │ 55.72602844238281 │
│ bldg_3e9e4e15-3683…  │ POLYGON ((139.74477272466495 35.66700668755646, 139.7444687328894 35.66…  │ 55.72602844238281 │
│ bldg_660a6fe6-dbd0…  │ POLYGON ((139.74373599923877 35.667795773549145, 139.74374646925907 35.…  │ 55.72602844238281 │
│ bldg_6d4b9ab2-db75…  │ POLYGON ((139.73544700956978 35.680677084621124, 139.73511680749166 35.…  │  55.2908935546875 │
│ bldg_320b7641-aa4e…  │ POLYGON

## クエリの結果をGeoJSONで出力

In [21]:
duckdb.sql("copy ({}) to 'test.geojson' with (format gdal, driver 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES')".format(query_01))


In [38]:
# change to DF
df = duckdb.sql('select gml_id, ST_AsText(geom) geom_txt from building_polygon limit 500').df()
df

,gml_id,geom_txt
0,bldg_fc50c7d9-76ac-4576-bfbd-f37c74410928,POLYGON ((139.7123070884317 35.541587725969336...
1,bldg_2e11a57e-c315-4351-8cca-22649d9763dc,POLYGON ((139.70715471131086 35.54049347329776...
2,bldg_72de059a-ddf0-4959-be00-db8108ca4a3a,POLYGON ((139.70650225272118 35.54134514528564...
3,bldg_2297d74f-b3d5-410a-b62c-a41f6913758c,"POLYGON ((139.706557904104 35.541302207399234,..."
4,bldg_f6edafc1-b722-4b15-90e3-393cbb64d998,POLYGON ((139.71058239366005 35.54154636660162...
...,...,...
495,bldg_f9dd69da-d3b8-43a9-8b6e-9198c70d6fa4,POLYGON ((139.70707248053844 35.54271892419017...
496,bldg_1d96d223-40c5-4804-bde5-78e9e5cfe35b,POLYGON ((139.70470434678694 35.54861855219195...
497,bldg_38fba4dd-4ab1-478d-a930-05a6c1506107,POLYGON ((139.7063953013518 35.548537596388506...
498,bldg_23e83c98-cb5c-4ebc-b0e8-bc577894ad12,POLYGON ((139.71235882592032 35.54811457430077...


In [39]:
# change to GDF
df["geom"] = gpd.GeoSeries.from_wkt(df["geom_txt"])
gdf = gpd.GeoDataFrame(df, geometry="geom")
gdf

,gml_id,geom_txt,geom
0,bldg_fc50c7d9-76ac-4576-bfbd-f37c74410928,POLYGON ((139.7123070884317 35.541587725969336...,"POLYGON ((139.71231 35.54159, 139.71230 35.541..."
1,bldg_2e11a57e-c315-4351-8cca-22649d9763dc,POLYGON ((139.70715471131086 35.54049347329776...,"POLYGON ((139.70715 35.54049, 139.70711 35.540..."
2,bldg_72de059a-ddf0-4959-be00-db8108ca4a3a,POLYGON ((139.70650225272118 35.54134514528564...,"POLYGON ((139.70650 35.54135, 139.70645 35.541..."
3,bldg_2297d74f-b3d5-410a-b62c-a41f6913758c,"POLYGON ((139.706557904104 35.541302207399234,...","POLYGON ((139.70656 35.54130, 139.70652 35.541..."
4,bldg_f6edafc1-b722-4b15-90e3-393cbb64d998,POLYGON ((139.71058239366005 35.54154636660162...,"POLYGON ((139.71058 35.54155, 139.71059 35.541..."
...,...,...,...
495,bldg_f9dd69da-d3b8-43a9-8b6e-9198c70d6fa4,POLYGON ((139.70707248053844 35.54271892419017...,"POLYGON ((139.70707 35.54272, 139.70707 35.542..."
496,bldg_1d96d223-40c5-4804-bde5-78e9e5cfe35b,POLYGON ((139.70470434678694 35.54861855219195...,"POLYGON ((139.70470 35.54862, 139.70468 35.548..."
497,bldg_38fba4dd-4ab1-478d-a930-05a6c1506107,POLYGON ((139.7063953013518 35.548537596388506...,"POLYGON ((139.70640 35.54854, 139.70639 35.548..."
498,bldg_23e83c98-cb5c-4ebc-b0e8-bc577894ad12,POLYGON ((139.71235882592032 35.54811457430077...,"POLYGON ((139.71236 35.54811, 139.71235 35.548..."


In [40]:
# visualize data with leafmap
m = leafmap.Map()
m.add_gdf(gdf, layer_name="Cable lines")
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [48]:
import leafmap.deck as leafmap
m = leafmap.Map(center=[40, -100], zoom=2, height=600)
m

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "latitude": 40,
    "longitude": -100,
    "zoom": 2
  },
  "layers": [],
  "mapProvider": "carto",
  "mapStyle": "https://basemaps.cartocdn.com/gl/positron-gl-style/style.json",
  "views": [
    {
      "@@type": "MapView",
      "controller": true
    }
  ]
}

In [49]:
m.static_map(width=950, height=600, read_only=True)

AttributeError: ignored